In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import textwrap

class MysterySolver:
    def __init__(self):
        # Define the Bayesian Network structure
        self.model = BayesianNetwork([
            ('Motive', 'DetectiveDanGuilty'),
            ('Motive', 'ProfessorPennyGuilty'),
            ('Motive', 'ChefCharlieGuilty'),
            ('AlibiDan', 'DetectiveDanGuilty'),
            ('AlibiPenny', 'ProfessorPennyGuilty'),
            ('Evidence', 'DetectiveDanGuilty'),
            ('Evidence', 'ProfessorPennyGuilty'),
            ('WitnessTestimony', 'ChefCharlieGuilty'),
            ('WeaponFound', 'ChefCharlieGuilty'),
            ('Behavior', 'ChefCharlieGuilty')
        ])
        
        # Define CPDs (Conditional Probability Distributions)
        cpd_motive = TabularCPD(variable='Motive', variable_card=2, values=[[0.6], [0.4]])
        cpd_alibi_dan = TabularCPD(variable='AlibiDan', variable_card=2, values=[[0.9], [0.1]])
        cpd_alibi_penny = TabularCPD(variable='AlibiPenny', variable_card=2, values=[[0.8], [0.2]])
        cpd_evidence = TabularCPD(variable='Evidence', variable_card=2, values=[[0.5], [0.5]])
        cpd_behavior = TabularCPD(variable='Behavior', variable_card=2, values=[[0.7], [0.3]])
        cpd_weapon = TabularCPD(variable='WeaponFound', variable_card=2, values=[[0.8], [0.2]])
        cpd_witness = TabularCPD(variable='WitnessTestimony', variable_card=2, values=[[0.6], [0.4]])

        # CPDs for each suspect
        cpd_dan_guilty = TabularCPD(
            variable='DetectiveDanGuilty', variable_card=2,
            values=[
                [0.9, 0.6, 0.4, 0.2, 0.8, 0.5, 0.3, 0.1],  # Not guilty
                [0.1, 0.4, 0.6, 0.8, 0.2, 0.5, 0.7, 0.9]   # Guilty
            ],
            evidence=['Motive', 'AlibiDan', 'Evidence'],
            evidence_card=[2, 2, 2]
        )
        
        cpd_penny_guilty = TabularCPD(
            variable='ProfessorPennyGuilty', variable_card=2,
            values=[
                [0.85, 0.5, 0.3, 0.1, 0.75, 0.4, 0.2, 0.05],  # Not guilty
                [0.15, 0.5, 0.7, 0.9, 0.25, 0.6, 0.8, 0.95]   # Guilty
            ],
            evidence=['Motive', 'AlibiPenny', 'Evidence'],
            evidence_card=[2, 2, 2]
        )

        cpd_chef_guilty = TabularCPD(
            variable='ChefCharlieGuilty', variable_card=2,
            values=[
                [0.85, 0.6, 0.4, 0.1, 0.75, 0.5, 0.2, 0.05],  # Not guilty
                [0.15, 0.4, 0.6, 0.9, 0.25, 0.5, 0.8, 0.95]   # Guilty
            ],
            evidence=['Motive', 'WitnessTestimony', 'WeaponFound', 'Behavior'],
            evidence_card=[2, 2, 2, 2]
        )
        
        # Add CPDs to the model
        self.model.add_cpds(cpd_motive, cpd_alibi_dan, cpd_alibi_penny, cpd_evidence, 
                            cpd_behavior, cpd_weapon, cpd_witness, cpd_dan_guilty, 
                            cpd_penny_guilty, cpd_chef_guilty)

        # Verify the model structure
        assert self.model.check_model()

        # Set up the inference engine
        self.inference = VariableElimination(self.model)

    def infer_suspects(self, motive, alibi_dan, alibi_penny, evidence, witness, weapon, behavior):
        # Map inputs to Bayesian network values
        motive_value = 1 if motive == 'yes' else 0
        alibi_dan_value = 1 if alibi_dan == 'no' else 0
        alibi_penny_value = 1 if alibi_penny == 'yes' else 0
        evidence_value = 1 if evidence == 'yes' else 0
        witness_value = 1 if witness == 'yes' else 0
        weapon_value = 1 if weapon == 'yes' else 0
        behavior_value = 1 if behavior == 'yes' else 0

        # Query the network for probabilities of each suspect being guilty
        result_dan = self.inference.query(variables=['DetectiveDanGuilty'], evidence={
            'Motive': motive_value, 'AlibiDan': alibi_dan_value, 'Evidence': evidence_value
        })
        result_penny = self.inference.query(variables=['ProfessorPennyGuilty'], evidence={
            'Motive': motive_value, 'AlibiPenny': alibi_penny_value, 'Evidence': evidence_value
        })
        result_chef = self.inference.query(variables=['ChefCharlieGuilty'], evidence={
            'Motive': motive_value, 'WitnessTestimony': witness_value, 
            'WeaponFound': weapon_value, 'Behavior': behavior_value
        })

        # Display results
        print("\n### Mystery Results ###")
        print("Probability of Detective Dan being Guilty:")
        print(result_dan)
        print("\nProbability of Professor Penny being Guilty:")
        print(result_penny)
        print("\nProbability of Chef Charlie being Guilty:")
        print(result_chef)

    def visualize_network(self):
        """Visualizes the Bayesian Network structure."""
        plt.figure(figsize=(8, 6))
        G = nx.DiGraph()
        G.add_edges_from(self.model.edges())
        pos = nx.spring_layout(G)
        nx.draw(G, pos, with_labels=True, node_size=2000, node_color='lightblue', font_size=10, font_weight='bold')
        plt.title("Bayesian Network Structure")
        plt.show()


# Main application code
def main():
    # Updated story and introduction
    story = textwrap.dedent("""\
        Welcome to the Mystery Solver!

        A crime has rocked the small town of Greenfield. Three suspects are under scrutiny:
        - Detective Dan: Known for his financial troubles and secrecy.
        - Professor Penny: A scholar with a history of feuds.
        - Chef Charlie: A debt-ridden chef with a murky alibi.

        As the lead investigator, you must analyze motives, alibis, and evidence to uncover the truth. 
        Choose wisely—an innocent life hangs in the balance!
    """)
    print(story)

    # Initialize the mystery solver
    solver = MysterySolver()

    # Collect evidence from the user
    motive = input("Do you believe there was a motive (yes/no)? ").strip().lower()
    alibi_dan = input("Detective Dan claims he was at the café. Does he have a solid alibi (yes/no)? ").strip().lower()
    alibi_penny = input("Professor Penny says she was in her office. Is her alibi questionable (yes/no)? ").strip().lower()
    evidence = input("Was there physical evidence linking one of them to the crime scene (yes/no)? ").strip().lower()
    witness = input("Did a witness see one of the suspects near the crime scene (yes/no)? ").strip().lower()
    weapon = input("Was a weapon found near the scene of the crime (yes/no)? ").strip().lower()
    behavior = input("Did any of the suspects behave suspiciously during the investigation (yes/no)? ").strip().lower()

    # Run inference on the suspects
    solver.infer_suspects(motive, alibi_dan, alibi_penny, evidence, witness, weapon, behavior)

    # Visualize the network
    solver.visualize_network()

if __name__ == "__main__":
    main()
